In [43]:
# import the necessary packages
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

In [44]:
class LeNet(Module):
	def __init__(self, numChannels, classes):
		# call the parent constructor
		super(LeNet, self).__init__()
		# initialize first set of CONV => RELU => POOL layers
		self.conv1 = Conv2d(in_channels=numChannels, out_channels=20,
			kernel_size=(5, 5))
		self.relu1 = ReLU()
		self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		# initialize second set of CONV => RELU => POOL layers
		self.conv2 = Conv2d(in_channels=20, out_channels=50,
			kernel_size=(5, 5))
		self.relu2 = ReLU()
		self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		# initialize first (and only) set of FC => RELU layers
		self.fc1 = Linear(in_features=186050, out_features=64)
		self.relu3 = ReLU()
		# initialize our softmax classifier
		self.fc2 = Linear(in_features=64, out_features=classes)
		self.logSoftmax = LogSoftmax(dim=1)
	def forward(self, x):
		# pass the input through our first set of CONV => RELU =>
		# POOL layers
		x = self.conv1(x)
		x = self.relu1(x)
		x = self.maxpool1(x)
		# pass the output from the previous layer through the second
		# set of CONV => RELU => POOL layers
		x = self.conv2(x)
		x = self.relu2(x)
		x = self.maxpool2(x)
		# flatten the output from the previous layer and pass it
		# through our only set of FC => RELU layers
		x = flatten(x, 1)
		x = self.fc1(x)
		x = self.relu3(x)
		# pass the output to our softmax classifier to get our output
		# predictions
		x = self.fc2(x)
		output = self.logSoftmax(x)
		# return the output predictions
		return output

In [45]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.optim import Adam
from torch import nn
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time
import os

In [46]:
# Simulate command-line arguments inside the notebook
args_list = ['-m', 'model_output.h5', '-p', 'plot_output.png']

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-m", "--model", type=str, required=True,
	help="path to output trained model", default='model_output.h5')
ap.add_argument("-p", "--plot", type=str, required=True,
	help="path to output loss/accuracy plot", default='plot_output.png')
args = vars(ap.parse_args(args_list))

In [47]:
# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10
# define the train and val splits
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [48]:
# new class to replace KMNIST with our data
class CustomFolderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        
        # Map folder names to labels
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        
        # Collect all (image_path, label) pairs
        self.samples = []
        for cls_name in self.classes:
            cls_folder = os.path.join(root_dir, cls_name)
            for fname in os.listdir(cls_folder):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                    path = os.path.join(cls_folder, fname)
                    label = self.class_to_idx[cls_name]
                    self.samples.append((path, label))
        
        self.targets = [label for _, label in self.samples]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [49]:
# load the KMNIST dataset
print("[INFO] loading the KMNIST dataset...")
trainData = CustomFolderDataset(root_dir='./Hey-Waldo/256', transform=ToTensor())
testData = CustomFolderDataset(root_dir='./Hey-Waldo/256', transform=ToTensor())
# calculate the train/validation split
print("[INFO] generating the train/validation split...")
numTrainSamples = int(len(trainData) * TRAIN_SPLIT)
numValSamples = len(trainData) - numTrainSamples
(trainData, valData) = random_split(trainData,
	[numTrainSamples, numValSamples],
	generator=torch.Generator().manual_seed(42))

[INFO] loading the KMNIST dataset...
[INFO] generating the train/validation split...


In [50]:
# initialize the train, validation, and test data loaders
trainDataLoader = DataLoader(trainData, shuffle=True,
	batch_size=BATCH_SIZE)
valDataLoader = DataLoader(valData, batch_size=BATCH_SIZE)
testDataLoader = DataLoader(testData, batch_size=BATCH_SIZE)
# calculate steps per epoch for training and validation set
trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
valSteps = len(valDataLoader.dataset) // BATCH_SIZE

In [51]:
# Unused code that we could make a custom CNN from (sourced from HW 4)
# Assumed start of 64 pixels
def createCnnSequential():
    return nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Flatten(),
        nn.Linear(4096,256),
        nn.ReLU(),
        nn.Linear(256, 2)
    )

In [56]:
# initialize the model
from sklearn.utils import compute_class_weight

print("[INFO] initializing the model...")
# model = createCnnSequential().to(device)
model = LeNet(
	numChannels=3,
	classes=len(trainData.dataset.classes)).to(device)

# Access correct targets from the split subset
full_targets = trainData.dataset.targets
train_targets = [full_targets[i] for i in trainData.indices]

# Compute weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_targets),
    y=train_targets
)

class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = nn.CrossEntropyLoss(weight=class_weights)
# initialize a dictionary to store training history
H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}
# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()

# loop over our epochs
for e in range(0, EPOCHS):
	# set the model in training mode
	model.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalValLoss = 0
	# initialize the number of correct predictions in the training
	# and validation step
	trainCorrect = 0
	valCorrect = 0
	# loop over the training set
	for (x, y) in trainDataLoader:
		# send the input to the device
		(x, y) = (x.to(device), y.to(device))
		# perform a forward pass and calculate the training loss
		pred = model(x)
		loss = lossFn(pred, y)
		# zero out the gradients, perform the backpropagation step,
		# and update the weights
		opt.zero_grad()
		loss.backward()
		opt.step()
		# add the loss to the total training loss so far and
		# calculate the number of correct predictions
		totalTrainLoss += loss
		trainCorrect += (pred.argmax(1) == y).type(
			torch.float).sum().item()
	# switch off autograd for evaluation
	with torch.no_grad():
		# set the model in evaluation mode
		model.eval()
		# loop over the validation set
		for (x, y) in valDataLoader:
			# send the input to the device
			(x, y) = (x.to(device), y.to(device))
			# make the predictions and calculate the validation loss
			pred = model(x)
			totalValLoss += lossFn(pred, y)
			# calculate the number of correct predictions
			valCorrect += (pred.argmax(1) == y).type(
				torch.float).sum().item()
	# calculate the average training and validation loss
	avgTrainLoss = totalTrainLoss / trainSteps
	avgValLoss = totalValLoss / valSteps
	# calculate the training and validation accuracy
	trainCorrect = trainCorrect / len(trainDataLoader.dataset)
	valCorrect = valCorrect / len(valDataLoader.dataset)
	# update our training history
	H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
	H["train_acc"].append(trainCorrect)
	H["val_loss"].append(avgValLoss.cpu().detach().numpy())
	H["val_acc"].append(valCorrect)
	# print the model training and validation information
	print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
	print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
		avgTrainLoss, trainCorrect))
	print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
		avgValLoss, valCorrect))

# finish measuring how long training took
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
	endTime - startTime))
# we can now evaluate the network on the test set
print("[INFO] evaluating network...")
# turn off autograd for testing evaluation
with torch.no_grad():
	# set the model in evaluation mode
	model.eval()
	
	# initialize a list to store our predictions
	preds = []
	# loop over the test set
	for (x, y) in testDataLoader:
		# send the input to the device
		x = x.to(device)
		# make the predictions and add them to the list
		pred = model(x)
		preds.extend(pred.argmax(axis=1).cpu().numpy())

[INFO] initializing the model...
[INFO] training the network...
[INFO] EPOCH: 1/10
Train loss: 9.412308, Train accuracy: 0.5359
Val loss: 1.227269, Val accuracy: 0.9000

[INFO] EPOCH: 2/10
Train loss: 0.971994, Train accuracy: 0.9030
Val loss: 1.309961, Val accuracy: 0.9000

[INFO] EPOCH: 3/10
Train loss: 0.905338, Train accuracy: 0.9030
Val loss: 1.229573, Val accuracy: 0.9000

[INFO] EPOCH: 4/10
Train loss: 0.929635, Train accuracy: 0.9030
Val loss: 1.220439, Val accuracy: 0.9000

[INFO] EPOCH: 5/10
Train loss: 0.874210, Train accuracy: 0.9030
Val loss: 1.204914, Val accuracy: 0.9125

[INFO] EPOCH: 6/10
Train loss: 0.809571, Train accuracy: 0.9198
Val loss: 1.109525, Val accuracy: 0.9125

[INFO] EPOCH: 7/10
Train loss: 0.687188, Train accuracy: 0.9451
Val loss: 0.978368, Val accuracy: 0.9375

[INFO] EPOCH: 8/10
Train loss: 0.824218, Train accuracy: 0.6920
Val loss: 0.975936, Val accuracy: 0.8750

[INFO] EPOCH: 9/10
Train loss: 0.538658, Train accuracy: 0.9409
Val loss: 0.931014, Val 

In [57]:
# generate a classification report
print(classification_report(testData.targets,
	np.array(preds), target_names=testData.classes))

              precision    recall  f1-score   support

    notwaldo       0.97      1.00      0.98       286
       waldo       0.96      0.74      0.84        31

    accuracy                           0.97       317
   macro avg       0.97      0.87      0.91       317
weighted avg       0.97      0.97      0.97       317



In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["val_loss"], label="val_loss")
plt.plot(H["train_acc"], label="train_acc")
plt.plot(H["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

plt.savefig(args["plot"])
# serialize the model to disk
torch.save(model, args["model"])

In [59]:
import tensorflow as tf
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt

def load_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


In [ ]:
def sliding_window(image, step=32, window_size=(128, 128)):
    for y in range(0, image.shape[0] - window_size[1], step):
        for x in range(0, image.shape[1] - window_size[0], step):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

In [ ]:
def detect_waldo_in_image(model, full_image, window_size=(128, 128), step=32, threshold=0.9):
    boxes = []

    for (x, y, window) in sliding_window(full_image, step=step, window_size=window_size):
        if window.shape[0] != window_size[1] or window.shape[1] != window_size[0]:
            continue  # Skip incomplete patches at the edge

        input_img = tf.image.resize(window, window_size)
        input_img = tf.expand_dims(input_img / 255.0, axis=0)  # Normalize

        prediction = model.predict(input_img, verbose=0)[0][0]

        if prediction >= threshold:
            boxes.append((x, y, prediction))

    return boxes


In [ ]:
def draw_detections(image, boxes, window_size=(128, 128)):
    img_copy = image.copy()
    for (x, y, score) in boxes:
        cv2.rectangle(img_copy, (x, y), (x + window_size[0], y + window_size[1]), (255, 0, 0), 2)
        cv2.putText(img_copy, f"{score:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)
    return img_copy


In [ ]:
img_path = "Hey-Waldo/original-images/1.jpg"
# img_path = "Hey-Waldo/256/waldo/1_1_1.jpg"
image = load_image(img_path)

In [ ]:
x, y = 0, 0
test_window = image[y:y+128, x:x+128]  # Pick a spot you know Waldo is in
test_window = cv2.resize(test_window, (128, 128))
test_window = test_window.astype("float32") / 255.0
test_window = np.expand_dims(test_window, axis=0)

prediction = model.predict(test_window)
print("Prediction score:", prediction)

In [ ]:
# Example threshold values
threshold = 0.7

# Multiply by 100000 to shift the decimal point
# threshold_display = int(threshold * 1000000)

# Display as 15 instead of 0.00015
# print(f"Threshold: {threshold_display}")

boxes = detect_waldo_in_image(model, image, window_size=(256, 256), step=128, threshold=threshold)
# for some reason every single bounding box has the same threshold value???

result_image = draw_detections(image, boxes)

# plt.figure(figsize=(12, 12))
# plt.title(f"Threshold: {threshold_display}")
plt.imshow(result_image)
plt.axis('off')
plt.show()